In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn.functional as F
import gymnasium as gym

import json
import pandas as pd
import logging

from survey_ops.plotting import plot_schedule_from_file
from survey_ops.coreRL.agents import Agent
from survey_ops.utils.sys_utils import seed_everything
from survey_ops.algorithms import setup_algorithm
from survey_ops.utils.sys_utils import setup_logger, get_device
from survey_ops.coreRL.data_processing import load_raw_data_to_dataframe
from survey_ops.coreRL.environments import OfflineDECamTestingEnv
from survey_ops.coreRL.offline_dataset import OfflineDELVEDataset
from survey_ops.utils.config import save_config, load_global_config, dict_to_nested
import logging
logger = logging.getLogger(__name__)


from survey_ops.utils import units
from pathlib import Path

%reload_ext autoreload
%autoreload 2

In [2]:
from survey_ops.utils import ephemerides

In [3]:
trained_model_dir = '../experiment_results/nside16/bc-azel-grid-4years-ang-dist/'
specific_years = [2016, 2017]
specific_months = [12]
specific_days = [21, 5]#, 21]

PROJECT_ROOT = Path('../')

In [4]:
# Get configs
gcfg = load_global_config(PROJECT_ROOT / 'configs' / 'global_config.json')
config_path = Path(trained_model_dir) / "config.json"
with open(config_path, "r") as f:
    cfg = json.load(f)

results_outdir = cfg['metadata']['outdir'] + '/' + 'evaluation_1' + '/'
if not os.path.exists(results_outdir):
    os.makedirs(results_outdir)

In [5]:
device = get_device()
seed_everything(10)

In [6]:
df = load_raw_data_to_dataframe(Path(gcfg['paths']['FITS_DIR']) / Path(gcfg['files']['DECFITS']))
nside = cfg['data']['nside']
test_dataset = OfflineDELVEDataset(
    df=df,
    cfg=cfg,
    gcfg=gcfg,
    specific_years=specific_years,
    specific_months=specific_months,
    specific_days=specific_days,
    specific_filters=None
    ) 

/home/hurra/Projects/survey-ops/survey_ops/coreRL/data_processing.py:41: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = ~df['object'].str.contains(pattern, case=False, na=False)
Calculating sun and moon ra/dec and az/el: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 533/533 [00:00<00:00, 1237.95it/s]
Calculating bin features for all healpix bins and timestamps: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 533/533 [00:00<00:00, 10210.34it/s]


In [7]:
from survey_ops.coreRL.data_processing import normalize_noncyclic_features, get_zenith_features

In [8]:
with open(gcfg['paths']['LOOKUP_DIR'] + '/' + gcfg['files']['FIELD2NVISITS'], 'r') as f:
    field2nvisits = json.load(f)
# with open(glob_cfg.get('paths.lookup_dir') + '/' + cfg.get('paths')['FIELD2NAME'], 'r') as f:
#     field2name = json.load(f)
with open(gcfg['paths']['LOOKUP_DIR'] + '/' + gcfg['files']['FIELD2RADEC'], 'r') as f:
    field2radec = json.load(f)
with open(f'../data/lookups/nside{nside}_bin2azel.json', 'r') as f:
    bin2azel = json.load(f)
with open(f'../data/lookups/nside{nside}_bin2radec.json', 'r') as f:
    bin2radec = json.load(f)

field2nvisits = {int(k): v for k, v in field2nvisits.items()}
field_ids = np.array(list(field2nvisits.keys()))
field2radec = {int(k): v for k, v in field2radec.items()}
field_radecs = np.array(list(field2radec.values()))

In [10]:
algorithm = setup_algorithm(algorithm_name=cfg['model']['algorithm'], 
                            num_actions=cfg['data']['num_actions'],
                            n_global_features = test_dataset.states.shape[-1],
                            n_bin_features=0 if test_dataset.bin_states is None else test_dataset.bin_states.shape[-1],
                            grid_network=cfg['model']['grid_network'],
                            loss_fxn=cfg['model']['loss_function'],
                            hidden_dim=cfg['train']['hidden_dim'], lr=cfg['train']['lr'], lr_scheduler=cfg['train']['lr_scheduler'], 
                            device=device, lr_scheduler_kwargs=cfg['train']['lr_scheduler_kwargs'], lr_scheduler_epoch_start=cfg['train']['lr_scheduler_epoch_start'], 
                            lr_scheduler_num_epochs=cfg['train']['lr_scheduler_num_epochs'],
                            gamma=cfg['model']['gamma'], 
                            tau=cfg['model']['tau'],
                            activation=cfg['model']['activation']
                            )
agent = Agent(
    algorithm=algorithm,
    train_outdir=trained_model_dir,
)
agent.load(trained_model_dir + 'best_weights.pt')

In [11]:
env_name = 'OfflineDECamEnv-v0'
gym.register(
id=f"gymnasium_env/{env_name}",
entry_point=OfflineDECamTestingEnv,
)

# Creat env
global_pd_nightgroup = test_dataset._df.groupby('night')
if len(cfg['data']['additional_bin_features']) > 0:
    bin_pd_nightgroup = test_dataset._bin_df.groupby('night')
else:
    bin_pd_nightgroup = None
env = gym.make(id=f"gymnasium_env/{env_name}", cfg=cfg, gcfg=gcfg, max_nights=None, global_pd_nightgroup=global_pd_nightgroup, bin_pd_nightgroup=bin_pd_nightgroup)


In [12]:
from survey_ops.coreRL.survey_logic import get_fields_in_bin

In [13]:
global_first_row = global_pd_nightgroup.head(1).iloc[0]
global_state = [global_first_row[feat_name] for feat_name in env.unwrapped.global_feature_names]
hpGrid = test_dataset.hpGrid
bin2fields_in_bin = env.unwrapped.bin2fields_in_bin
field_choice_method = 'random'

In [14]:
state, info = env.reset()
episode_reward = 0
terminated = False
truncated = False
num_nights = env.unwrapped.max_nights
glob_observations = {f'night-{i}': [] for i in range(num_nights)}
bin_observations = {f'night-{i}': [] for i in range(num_nights)}
rewards = {f'night-{i}': [] for i in range(num_nights)}
timestamps = {f'night-{i}': [] for i in range(num_nights)}
fields = {f'night-{i}': [] for i in range(num_nights)}
bins = {f'night-{i}': [] for i in range(num_nights)}

i = 0
reward = 0
night_idx = 0

In [15]:
t0 = info.get('timestamp')
while not (terminated or truncated):
    with torch.no_grad():
                
        timestamp = info.get('timestamp')
        print(f'Step {i}, Hours in night {(timestamp - t0)/3600:.2f}')
        
        glob_observations[f'night-{night_idx}'].append(state['global_state'])
        bin_observations[f'night-{night_idx}'].append(state['bins_state'])
        rewards[f'night-{night_idx}'].append(reward)
        timestamps[f'night-{night_idx}'].append(info.get('timestamp'))
        fields[f'night-{night_idx}'].append(info.get('field_id'))
        bins[f'night-{night_idx}'].append(info.get('bin'))

        action_mask = info.get('action_mask', None)
        action = agent.act(x_glob=state['global_state'], x_bin=state['bins_state'], action_mask=action_mask, epsilon=None)
        fields_in_bin = get_fields_in_bin(bin_num=action, is_azel=hpGrid.is_azel, timestamp=timestamp, field2nvisits=field2nvisits, field_ids=field_ids, field_radecs=field_radecs, hpGrid=hpGrid, visited=info.get('visited'), bin2fields_in_bin=bin2fields_in_bin)
        field_id = agent.choose_field(obs=(state['global_state'], state['bins_state']), info=info, field2nvisits=field2nvisits, field2radec=field2radec, hpGrid=hpGrid, field_choice_method=field_choice_method, fields_in_bin=fields_in_bin)

        actions = {'bin': np.int32(action), 'field_id': np.int32(field_id), 'filter': np.array([0], dtype=np.float32)}
        state, reward, terminated, truncated, info = env.step(actions)
        episode_reward += reward
        night_idx = info.get('night_idx')
        i += 1

Step 0, Hours in night 0.00
Step 1, Hours in night 0.03
Step 2, Hours in night 0.07
Step 3, Hours in night 0.10
Step 4, Hours in night 0.13
Step 5, Hours in night 0.17
Step 6, Hours in night 0.20
Step 7, Hours in night 0.23
Step 8, Hours in night 0.27
Step 9, Hours in night 0.30
Step 10, Hours in night 0.33
Step 11, Hours in night 0.37
Step 12, Hours in night 0.40
Step 13, Hours in night 0.43
Step 14, Hours in night 0.47
Step 15, Hours in night 0.50
Step 16, Hours in night 0.53
Step 17, Hours in night 0.57
Step 18, Hours in night 0.60
Step 19, Hours in night 0.63
Step 20, Hours in night 0.67
Step 21, Hours in night 0.70
Step 22, Hours in night 0.73
Step 23, Hours in night 0.77
Step 24, Hours in night 0.80
Step 25, Hours in night 0.83
Step 26, Hours in night 0.87
Step 27, Hours in night 0.90
Step 28, Hours in night 0.93
Step 29, Hours in night 0.97
Step 30, Hours in night 1.00
Step 31, Hours in night 1.03
Step 32, Hours in night 1.07
Step 33, Hours in night 1.10
Step 34, Hours in night 

In [16]:
np.array(bin_observations['night-0']).shape

(232, 1504, 1)

In [19]:
field2radec

{0: [5.256913279820678, -0.8890393748525456],
 1: [5.250846298919919, -0.9244491976569768],
 2: [5.244014989470491, -0.9598668709523833],
 3: [5.236278615102833, -0.9952758176015299],
 4: [5.283499412568068, -1.0129855757830781],
 5: [5.333349624764144, -1.030694462463554],
 6: [5.289091545229896, -0.9775686587970142],
 7: [5.336494286288334, -0.9952792303019941],
 8: [5.3392652146420305, -0.9598628607675388],
 9: [5.294017454882087, -0.9421568213008499],
 10: [5.341688937666313, -0.9244471591124105],
 11: [5.29839519035926, -0.9067450629262783],
 12: [5.3438472667972, -0.8890411158184746],
 13: [5.430784875913697, -0.8890336065393679],
 14: [5.43252999427174, -0.9244511803510072],
 15: [5.386994065235217, -0.9421562366155506],
 16: [5.38674593992033, -0.9775703604930349],
 17: [5.386446143551163, -1.0129788841907261],
 18: [5.439233176482785, -1.0306891310644652],
 19: [5.4367110186340195, -0.9952792140122544],
 20: [5.4344910017133845, -0.9598584305401209],
 21: [5.387215421853588, -

In [17]:
b_azels = np.array([bin2azel[str(bid)] for bid in bins['night-0'][1:]])
b_radecs = np.empty_like(b_azels)
for i, _t in enumerate(timestamps['night-0'][1:]):
    b_radecs[i] = ephemerides.topographic_to_equatorial(az=b_azels[i, 0], el=b_azels[i, 1], time=_t)

In [18]:
f_radecs = np.array([field2radec[fid] for fid in fields['night-0'][1:]])
plt.scatter(radecs[:, 0]/units.deg, radecs[:, 1]/units.deg, c=bins['night-0'][1:], s=10)
plt.scatter(b_radecs[:, 0]/units.deg, b_radecs[:, 1]/units.deg, c=bins['night-0'][1:], alpha=.5, s=100)

NameError: name 'radecs' is not defined